<a href="https://colab.research.google.com/github/tharakats/BankLoanEvaluator/blob/main/Model/BankLoanEvaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT3dBWyWxVw2rMsVnNaG36DWb33ecoq1Jd9i99ogk7S9mKDirDdg359NvLBmi5IlqLU9XFH4GLx6n_d/pub?gid=1809176138&single=true&output=csv')

In [3]:
X=dataset.iloc[:,3:-1].values
Y=dataset.iloc[:,-1].values

In [4]:
from sklearn.impute import SimpleImputer

In [5]:
imputer=SimpleImputer(missing_values=np.nan,strategy='mean')
X[:,4:12]=imputer.fit_transform(X[:,4:12])

In [6]:
from sklearn.model_selection import train_test_split

In [30]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler

In [31]:
gender=LabelEncoder()

In [32]:
gender_x=gender.fit(X_train[:,2])

X_train[:,2]=gender_x.transform(X_train[:,2])
X_test[:,2]=gender_x.transform(X_test[:,2])

In [22]:
X_train

array([[628, 'Germany', 1, ..., 0.0, 1.0, 124749.08],
       [626, 'France', 0, ..., 0.0, 0.0, 41104.82],
       [612, 'Germany', 0, ..., 1.0, 1.0, 45750.21],
       ...,
       [672, 'France', 0, ..., 1.0, 1.0, 92027.69],
       [663, 'France', 0, ..., 1.0, 1.0, 101168.9],
       [763, 'Germany', 1, ..., 1.0, 0.0, 33462.94]], dtype=object)

In [33]:
X_test

array([[550, 'France', 1, ..., 1.0, 1.0, 97057.28],
       [680, 'France', 1, ..., 1.0, 0.0, 66526.01],
       [531, 'France', 0, ..., 0.0, 1.0, 90537.47],
       ...,
       [567, 'Spain', 1, ..., 0.0, 1.0, 161571.79],
       [621, 'Germany', 0, ..., 1.0, 1.0, 165257.31],
       [655, 'Germany', 1, ..., 1.0, 1.0, 49025.79]], dtype=object)

In [24]:
country=LabelEncoder()

In [34]:
country_x=country.fit(X_train[:,1])
X_train[:,1]=country_x.transform(X_train[:,1])
X_test[:,1]=country_x.transform(X_test[:,1])

In [35]:
X_test

array([[550, 0, 1, ..., 1.0, 1.0, 97057.28],
       [680, 0, 1, ..., 1.0, 0.0, 66526.01],
       [531, 0, 0, ..., 0.0, 1.0, 90537.47],
       ...,
       [567, 2, 1, ..., 0.0, 1.0, 161571.79],
       [621, 1, 0, ..., 1.0, 1.0, 165257.31],
       [655, 1, 1, ..., 1.0, 1.0, 49025.79]], dtype=object)